In [4]:
from selenium import webdriver  
import time  
import urllib  
from slugify import slugify
import requests
import random
import os

In [5]:

"""
some variables needed to be set up by users
"""
conference = 'neurips'
conference_url = "https://papers.nips.cc/paper/2019" # the conference url to download papers from
chromedriver_path = 'C:\\Users\\xiaoyang\\Downloads\\chromedriver.exe' # the chromedriver.exe path
root = r'E:\OneDrive\academics\papers\conferences\NeurIPS-2019-ALL'.replace('\\','/') # file path to save the downloaded papers

os.makedirs(root, exist_ok=True)
print(root)

E:/OneDrive/academics/papers/conferences/NeurIPS-2019-ALL


In [6]:
"""
conference urls examples

cvpr: https://openaccess.thecvf.com/CVPR2020 (CVPR 2020)
eccv: https://openaccess.thecvf.com/ECCV2018 (ECCV 2018)
eccv: https://www.ecva.net/papers.php (ECCV 2020) (changed in 2020)
iccv: https://openaccess.thecvf.com/ICCV2019 (ICCV 2019)
icml: http://proceedings.mlr.press/v119/ (ICML 2020)
neurips: https://papers.nips.cc/paper/2020 (NeurIPS 2020)
iclr: https://openreview.net/group?id=ICLR.cc/2021/Conference (ICLR 2021)
siggraph: https://dl.acm.org/toc/tog/2020/39/4 (SIGGRAPH 2021)

"""




'\nconference urls examples\n\ncvpr: https://openaccess.thecvf.com/CVPR2020 (CVPR 2020)\neccv: https://openaccess.thecvf.com/ECCV2018 (ECCV 2018)\neccv: https://www.ecva.net/papers.php (ECCV 2020) (changed in 2020)\niccv: https://openaccess.thecvf.com/ICCV2019 (ICCV 2019)\nicml: http://proceedings.mlr.press/v119/ (ICML 2020)\nneurips: https://papers.nips.cc/paper/2020 (NeurIPS 2020)\niclr: https://openreview.net/group?id=ICLR.cc/2021/Conference (ICLR 2021)\nsiggraph: https://dl.acm.org/toc/tog/2020/39/4 (SIGGRAPH 2021)\n\n'

In [7]:
driver = webdriver.Chrome(chromedriver_path)  
driver.get(conference_url)

from retrieve_titles_urls_from_websites import *
retrieve = globals()['retrieve_from_'+conference]
print('Retrieving pdf urls. This could take some time...')
pdfurllist, pdfnamelist = retrieve(driver)



Retrieving pdf urls. This could take some time...


In [8]:
# check the retrieved urls
print('The first 5 pdf urls:\n')
for i in range(5):
    print(pdfurllist[i])
print('\nThe last 5 pdf urls:\n')
for i in range(5):
    print(pdfurllist[-(i+1)])


The first 5 pdf urls:

https://papers.nips.cc/paper/2019/file/00989c20ff1386dc386d8124ebcba1a5-Paper.pdf
https://papers.nips.cc/paper/2019/file/00ac8ed3b4327bdd4ebbebcb2ba10a00-Paper.pdf
https://papers.nips.cc/paper/2019/file/00e26af6ac3b1c1c49d7c3d79c60d000-Paper.pdf
https://papers.nips.cc/paper/2019/file/0118a063b4aae95277f0bc1752c75abf-Paper.pdf
https://papers.nips.cc/paper/2019/file/01386bd6d8e091c2ab4c7c7de644d37b-Paper.pdf

The last 5 pdf urls:

https://papers.nips.cc/paper/2019/file/ffedf5be3a86e2ee281d54cdc97bc1cf-Paper.pdf
https://papers.nips.cc/paper/2019/file/ffe10334251de1dc98339d99ae4743ba-Paper.pdf
https://papers.nips.cc/paper/2019/file/fface8385abbf94b4593a0ed53a0c70f-Paper.pdf
https://papers.nips.cc/paper/2019/file/fecc3a370a23d13b1cf91ac3c1e1ca92-Paper.pdf
https://papers.nips.cc/paper/2019/file/febefe1cc5c87748ea02036dbe9e3d67-Paper.pdf


In [9]:
# check the retrieved paper titles
print('The first 5 pdf titles:\n')
for i in range(5):
    print(pdfnamelist[i])
print('\nThe last 5 pdf titles:\n')
for i in range(5):
    print(pdfnamelist[-(i+1)])


The first 5 pdf titles:

Compositional Plan Vectors
Learning to Propagate for Graph Meta-Learning
XNAS: Neural Architecture Search with Expert Advice
Multi-resolution Multi-task Gaussian Processes
Deep Equilibrium Models

The last 5 pdf titles:

Flow-based Image-to-Image Translation with Feature Disentanglement
Efficient Rematerialization for Deep Networks
Fully Dynamic Consistent Facility Location
Adaptively Aligned Image Captioning via Adaptive Attention Time
Discrete Object Generation with Reversible Inductive Construction


In [10]:
print('The number of papers is ', len(pdfnamelist))
assert len(pdfnamelist)==len(pdfurllist), 'The number of titles and the number of urls are not matched. \
                                            You might solve the problem by checking the HTML code in the \
                                            website yourself or you could ask the author by raising an issue.'

The number of papers is  1428


In [11]:
# download the papers one by one. The files are named after the titles (guaranteed to be valid file name after processed by slugify.)
print('Start downloading')
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:57.0) Gecko/20100101 Firefox/57.0'}
for i, url in enumerate(pdfurllist):
    if url != None :      
        pdfname = slugify(pdfnamelist[i])
        if os.path.isfile(root+'/'+pdfname+".pdf"):
            print('existed', i, '\t', pdfnamelist[i], '\t', pdfurllist[i])
        else:
            print(i, '\t', pdfnamelist[i], '\t', pdfurllist[i])
            data = requests.get(pdfurllist[i], timeout=80, headers=headers).content
            
            with open(root+'/'+pdfname+".pdf", 'wb')  as f:
                f.write(data)  
            _ = time.sleep(random.uniform(4,5)) # for anti-anti-crawler purpose

Start downloading
0 	 Compositional Plan Vectors 	 https://papers.nips.cc/paper/2019/file/00989c20ff1386dc386d8124ebcba1a5-Paper.pdf


TypeError: 'module' object is not callable